In [9]:
from avaliacao_swaps import *
from queries import *
import re
from scipy.stats import norm
from swap_trades import trade_params_df 
from dateutil.relativedelta import relativedelta
warnings.simplefilter(action='ignore', category=Warning)
pd.set_option('display.float_format', '{:,.2f}'.format)
bbg_date = ql.TARGET().advance(ql.Date.todaysDate(), -1, ql.Days)
ql.Settings.instance().evaluationDate = bbg_date
curve_date = bbg_date.to_date()
curve_date_str = curve_date.strftime('%Y-%m-%d')
db_paths = [r"\\igcp-fs\NCF-GER\Projetos 010-01-01\1_DESENVOLVIMENTO_PROJECTOS_INTERNOS\BLOOMBERG_DATABASE\swaps.db"]
data_dict = fetch_data(queries, db_paths)
con_swaps = sqlite3.connect(db_paths[0])
df_3month = data_dict.get("query_3month")[0]
df_ois = data_dict.get("query_ois")[0]
df_fixings_estr = data_dict.get("query_estr_fixings")[0]
df_fixings_euri = data_dict.get("query_euribor_3m_fixings")[0]

In [10]:
ois_curve, discount_handle, tenors_ois, quotes_ois = build_estr_curve(curve_date_str,df_ois)
euribor_curve, tenors_euribor, quotes_euribor = build_euribor_curve(curve_date_str,df_3month, df_ois)

## Feed Zero Rates

In [12]:
zero_ois = extract_zero_rates(ois_curve, tenors_ois,bbg_date,curve_date_str)
zero_3m = extract_zero_rates(euribor_curve, tenors_euribor,bbg_date,curve_date_str)

In [9]:
zero_ois.to_sql("zero_rates_ois",conn, if_exists='append', index=False)
zero_3m.to_sql("zero_rates_3m",conn, if_exists='append', index=False)

23